In [4]:
#install latest Chrome driver to your machine https://sites.google.com/chromium.org/driver/
#install Beautiful Soup 
#install selenium

#imports
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
import time
import json 


#base product URL
URL = "https://e-katanalotis.gov.gr/product/"
min_id = 12
max_id = 22

#creates a list [min_id...max_id-1]
productids = range(min_id, max_id)

#final JSON
results = {}
results['fetch_date']=int(time.time())
results['data']=[]

#get product information
for productid in productids:
    print(productid)
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    # executable_path param is where the Chrome driver is installed
    browser = webdriver.Chrome(options=options, executable_path='/usr/bin/chromedriver')
    browser.get(URL+str(productid))

    #get dates from price chart
    try:
        date_data = browser.execute_script('return Highcharts.charts[0].series[0].data.map(x => x.category)')
    except exceptions.JavascriptException as e:
        continue
    #append year to date labels
    date_data = [s +'/2022' for s in date_data]

    #get prices from price chart
    try:
        price_data = browser.execute_script('return Highcharts.charts[0].series[0].data.map(x => x.y)')
    except exceptions.JavascriptException as e:
        continue
    #find product name
    html = browser.page_source
    soup = BeautifulSoup(html, features="html5lib")

    #test html rendering
    # print(soup.prettify())

    try:
        # pname = soup.find('p', attrs={'class':'product-name'}).get_text()
        pname = soup.find('p', attrs={'class':'product-category-name'}).get_text() 
    except AttributeError as e:
        continue
    browser.quit()

    #transform dates

    for i in range(len(date_data)-1):
        dparts = date_data[i].split("/")
        newd = '2022-'+dparts[1]+'-'+dparts[0]
        date_data[i]=newd
        
    #create a result object
    result = {}
    result['id']=productid
    result['name']=pname
    result['prices']=[]
    for i in range(len(date_data)-1):
        result['prices'].append({'date':date_data[i], 'price':price_data[i]})
    
    #append it to the list
    results['data'].append(result)

print('Done')

12


/tmp/ipykernel_7052/2750024456.py:32: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='/usr/bin/chromedriver')


13
14
15
16
17
18
19
20
21
Done


In [5]:
#inspect final JSON
json_object = json.dumps(results, indent = 2, ensure_ascii=False).encode('utf8')
print(json_object.decode())

{
  "fetch_date": 1670318130,
  "data": []
}


In [6]:
#save final JSON to file
with open("output_"+str(productids[0])+"_"+str(productids[len(productids)-1])+"_"+str(results['fetch_date'])+".json", "w", encoding='utf-8') as outfile:
    json.dump(results, outfile, ensure_ascii=False ,indent=2)